In [4]:
import os
import warnings

In [5]:
origen_libs = '/home/scopatz/origen22/libs'
tape9 = 'pwru50.lib'
lib = os.path.join(origen_libs, tape9)

In [7]:
from pyne import utils
utils.toggle_warnings()
warnings.simplefilter('ignore')
from pyne.origen22 import parse_tape9
from pyne import nucname
from pyne import rxname

In [8]:
t9 = parse_tape9(lib)

In [9]:
nlb = tuple(sorted(t9.keys()))

In [10]:
nlb

(219, 220, 221)

In [11]:
set(t9[220].keys())

{'_cards',
 '_subtype',
 '_type',
 'fiss_yields_present',
 'sigma_2n',
 'sigma_2n_x',
 'sigma_3n',
 'sigma_f',
 'sigma_gamma',
 'sigma_gamma_x',
 'title'}

In [12]:
t9[219]['sigma_gamma'][30060], t9[221]['sigma_gamma'][30060]

(0.0021870000000000001, 0.0021870000000000001)

In [13]:
XS_RXS = ['gamma', 'z_2n', 'z_3n', 'alpha', 'fission', 'proton', 'gamma_1', 'z_2n_1']
xs_2_o = {'gamma': 'gamma', 'z_2n': '2n', 'z_3n': '3n', 'alpha': 'alpha', 'fission': 'f', 'proton': 'p', 
          'gamma_1': 'gamma_x', 'z_2n_1': '2n_x'}
o_2_xs = {v: k for k, v in xs_2_o.items()}
sigma_maps = {}
for n in nlb:
    for rx in t9[n]:
        if not rx.startswith('sigma_'):
            continue
        for nuc in t9[n][rx]:
            if nuc not in sigma_maps:
                sigma_maps[nuc] = {}
            sigma_maps[nuc]['sigma_' + o_2_xs[rx[6:]]] = t9[n][rx][nuc]

In [14]:
# add absorption xs
for nuc in sigma_maps:
    sigma_maps[nuc]['sigma_a'] = sum(sigma_maps[nuc].values())

In [15]:
nucname.name(nucname.zzaaam_to_id(10010))

'H1'

In [23]:
sigma = {}
skip_rx = {'sigma_a', 'sigma_fission'}
for nuc in sigma_maps:
    n = nucname.zzaaam_to_id(int(nuc))
    name = nucname.name(n)
    for srx in sigma_maps[nuc]:
        if srx in skip_rx:
            child = None
        else:
            _, _, rx = srx.partition('_')
            print(name, rx, sigma_maps[nuc][srx])
            child = nucname.name(rxname.child(n, rx))
        sigma[srx + '_' + name] = [sigma_maps[nuc][srx], name, child]

Cd128 gamma 0.0
Cd128 proton 0.0
Cd128 gamma_1 0.0
Cd128 z_2n 0.0
Cd128 z_2n_1 0.0
Cd128 alpha 0.0
Sr93 gamma 0.0
Sr93 proton 0.0
Sr93 gamma_1 0.0
Sr93 z_2n 0.0
Sr93 z_2n_1 0.0
Sr93 alpha 0.0
Ni58 gamma 0.3603
Ni58 proton 0.0215
Ni58 gamma_1 0.0
Ni58 z_2n 2.437e-06
Ni58 z_2n_1 0.0
Ni58 alpha 0.00138
La140 gamma 2.297
La140 proton 0.0
La140 gamma_1 0.0
La140 z_2n 0.0
La140 z_2n_1 0.0
La140 alpha 0.0
Ta182 gamma 572.9
Ta182 proton 0.0
Ta182 gamma_1 0.0
Ta182 z_2n 0.0
Ta182 z_2n_1 0.0
Ta182 alpha 0.0
Cd129 gamma 0.0
Cd129 proton 0.0
Cd129 gamma_1 0.0
Cd129 z_2n 0.0
Cd129 z_2n_1 0.0
Cd129 alpha 0.0
Rn222 gamma 0.05623
Rn222 z_3n 0.0
Rn222 z_2n 0.0
Rn222 z_2n_1 0.0
Rn222 gamma_1 0.0
Sm157 gamma 0.0
Sm157 proton 0.0
Sm157 gamma_1 0.0
Sm157 z_2n 0.0
Sm157 z_2n_1 0.0
Sm157 alpha 0.0
Ni59 gamma 9.609
Ni59 proton 0.0
Ni59 gamma_1 0.0
Ni59 z_2n 0.0
Ni59 z_2n_1 0.0
Ni59 alpha 0.9371
La141 gamma 0.0
La141 proton 0.0
La141 gamma_1 0.0
La141 z_2n 0.0
La141 z_2n_1 0.0
La141 alpha 0.0
Cd130 gamma 0.0
C

RuntimeError: Not a Nuclide! 20030000 --> 0

In [ ]:
            child = rxname.child

In [18]:
sigma

{'sigma_a_Se80': 0.10193000000000001,
 'sigma_gamma_Kr87': 46.859999999999999,
 'sigma_proton_Zr94': 3.1080000000000001e-06,
 'sigma_proton_Xe144': 0.0,
 'sigma_fission_Cm247': 24.41,
 'sigma_gamma_1_Eu158': 0.0,
 'sigma_z_2n_1_Rh114': 0.0,
 'sigma_z_2n_Cu65': 0.00022010000000000001,
 'sigma_gamma_1_Ce154': 0.0,
 'sigma_fission_Np239': 0.41649999999999998,
 'sigma_proton_Sb128M': 0.0,
 'sigma_a_Tc101': 0.0,
 'sigma_a_Y96': 0.0,
 'sigma_z_2n_1_Dy163': 0.0,
 'sigma_z_2n_1_Th233': 0.0,
 'sigma_proton_S36': 0.0,
 'sigma_gamma_1_Rh114': 0.0,
 'sigma_proton_Sn121M': 0.0,
 'sigma_gamma_Rb90': 0.0,
 'sigma_a_Xe135': 207200.0,
 'sigma_fission_Cm245': 147.0,
 'sigma_alpha_Na23': 0.00016579999999999999,
 'sigma_proton_Ru102': 0.0,
 'sigma_gamma_1_In133': 0.0,
 'sigma_z_2n_1_Ba138': 0.0,
 'sigma_z_2n_H2': 0.0012080000000000001,
 'sigma_alpha_Cd129': 0.0,
 'sigma_z_2n_Y92': 0.0,
 'sigma_z_2n_Pd108': 0.0,
 'sigma_a_O18': 0.00049671999999999997,
 'sigma_gamma_Ag119': 0.0,
 'sigma_a_Ca43': 0.555864000

In [19]:
import json
with open('sigma.json', 'w') as f:
    json.dump(sigma, f, indent=' ', )